In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.metrics import mean_squared_error, mean_absolute_error, r2_score

typhoid = pd.read_csv('Cleaned_Typoid_Data.csv')

print(typhoid.head())
print(typhoid.info())

   year  week  month  adm3_pcode              adm3_en   tave   tmin   tmax  \
0  2015     1      1           1  City of Mandaluyong  24.90  23.05  27.55   
1  2015     1      1           2      City of Navotas  24.67  22.73  27.29   
2  2015     1      1           3   City of Muntinlupa  24.63  22.76  27.25   
3  2015     2      1           1  City of Mandaluyong  25.38  23.15  28.33   
4  2015     2      1           2      City of Navotas  25.27  23.12  28.03   

   heat_index   pr  ...  pharmacy_nearest  pop_count_total  pop_density_mean  \
0       25.77  2.2  ...            153.05         13551.19          35729.54   
1       25.38  0.0  ...            949.48         20156.26          44536.11   
2       25.46  6.4  ...            300.98         51784.80          11364.11   
3       26.46  0.0  ...            153.05         13551.19          35729.54   
4       26.20  0.0  ...            949.48         20156.26          44536.11   

   clinic_count  dentist_count  doctors_count  hos

In [2]:
typhoid = typhoid.drop(['adm3_en'], axis = 1)

print(typhoid.head())
print(typhoid.info())

   year  week  month  adm3_pcode   tave   tmin   tmax  heat_index   pr  \
0  2015     1      1           1  24.90  23.05  27.55       25.77  2.2   
1  2015     1      1           2  24.67  22.73  27.29       25.38  0.0   
2  2015     1      1           3  24.63  22.76  27.25       25.46  6.4   
3  2015     2      1           1  25.38  23.15  28.33       26.46  0.0   
4  2015     2      1           2  25.27  23.12  28.03       26.20  0.0   

   wind_speed  ...  pharmacy_nearest  pop_count_total  pop_density_mean  \
0        5.23  ...            153.05         13551.19          35729.54   
1        4.65  ...            949.48         20156.26          44536.11   
2        5.20  ...            300.98         51784.80          11364.11   
3        3.72  ...            153.05         13551.19          35729.54   
4        3.10  ...            949.48         20156.26          44536.11   

   clinic_count  dentist_count  doctors_count  hospital_count  optician_count  \
0           0.0        

In [3]:
X = typhoid.iloc[:, :-1]
y = typhoid.iloc[:, -1]

X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.3, random_state=42)

print(f"Train set size: {X_train.shape[0]}")
print(f"Test set size: {X_test.shape[0]}")
print(f"Total size: {X_train.shape[0] + X_test.shape[0]}")

Train set size: 1050
Test set size: 450
Total size: 1500


In [4]:
knn = KNeighborsRegressor()

param_grid_KNN = {
    'n_neighbors': np.arange(1, 31, 2),
    'weights': ['uniform', 'distance'],
    'algorithm': ['auto', 'ball_tree', 'kd_tree', 'brute'],
    'leaf_size': [10, 20, 30, 40, 50],
    'metric': ['minkowski', 'manhattan', 'euclidean', 'chebyshev'],
    'p': [1, 2]
}

grid_search = GridSearchCV(knn, param_grid_KNN, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_knn = grid_search.best_estimator_
print("Best Parameters: ", grid_search.best_params_)

y_pred = best_knn.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Best Parameters:  {'algorithm': 'kd_tree', 'leaf_size': 30, 'metric': 'chebyshev', 'n_neighbors': 7, 'p': 1, 'weights': 'distance'}
Mean Squared Error: 102.23515153935338
Root Mean Squared Error (RMSE): 10.111139972295575
Mean Absolute Error: 5.85661118801438
R-squared: 0.3172229929317092


In [5]:
decision_tree = DecisionTreeRegressor()

param_grid_DT = {
    'criterion': ['friedman_mse', 'poisson'],
    'max_depth': [5, 10, 15, 20, None],
    'min_samples_split': [2, 5, 10],
    'min_samples_leaf': [1, 2, 4],
    'max_features': ['sqrt', 'log2', None],
    'max_leaf_nodes': [None, 10, 20, 50, 100],
    'min_impurity_decrease': [0.0, 0.01, 0.05],
    'ccp_alpha': [0.0, 0.01, 0.05]
}

grid_search = GridSearchCV(decision_tree, param_grid_DT, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_dt = grid_search.best_estimator_
print("Best Parameters: ", grid_search.best_params_)

y_pred = best_dt.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Best Parameters:  {'ccp_alpha': 0.05, 'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': 20, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 4, 'min_samples_split': 2}
Mean Squared Error: 117.11215602885272
Root Mean Squared Error (RMSE): 10.821836998811834
Mean Absolute Error: 6.496608075957836
R-squared: 0.21786698429340934


In [6]:
random_forest = RandomForestRegressor()

param_grid_RFR = {
    'n_estimators': [100, 200],  # Reduced the number of estimators
    'max_depth': [None, 10, 20],  # Slightly fewer depth options
    'min_samples_split': [2, 5],  # Kept this as is
    'min_samples_leaf': [1, 2],  # Reduced leaf options
    'max_features': ['sqrt', 'log2'],  # Removed None
    'criterion': ['friedman_mse', 'poisson'],  # Kept both criteria
    'bootstrap': [True],  # Removed False
    'max_leaf_nodes': [None, 20, 50],  # Reduced node options
    'min_impurity_decrease': [0.0, 0.01],  # Kept only the lower values
    'ccp_alpha': [0.0, 0.01]  # Kept only the lower values
}

grid_search = GridSearchCV(random_forest, param_grid_RFR, cv=5, scoring='neg_mean_squared_error')
grid_search.fit(X_train, y_train)

best_rfr = grid_search.best_estimator_
print("Best Parameters: ", grid_search.best_params_)

y_pred = best_rfr.predict(X_test)

mse = mean_squared_error(y_test, y_pred)
rmse = np.sqrt(mse)
mae = mean_absolute_error(y_test, y_pred)
r2 = r2_score(y_test, y_pred)

print("Mean Squared Error:", mse)
print("Root Mean Squared Error (RMSE):", rmse)
print("Mean Absolute Error:", mae)
print("R-squared:", r2)

Best Parameters:  {'bootstrap': True, 'ccp_alpha': 0.01, 'criterion': 'friedman_mse', 'max_depth': None, 'max_features': 'log2', 'max_leaf_nodes': None, 'min_impurity_decrease': 0.0, 'min_samples_leaf': 1, 'min_samples_split': 2, 'n_estimators': 100}
Mean Squared Error: 67.93114935666333
Root Mean Squared Error (RMSE): 8.242035510519433
Mean Absolute Error: 5.146173421576926
R-squared: 0.5463221196811365
